###  Created by Luis A. Sanchez-Perez (alejand@umich.edu)

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy.stats import norm 
from scipy.special import logsumexp

In [ ]:
# Load dataset
dataset = datasets.load_iris()
print(dataset.feature_names, end="\n")
print(dataset.target_names)

In [ ]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(dataset.data,dataset.target)

### Using sklearn implementation

In [ ]:
# Fitting Naive Bayes
classifier = GaussianNB()
classifier.fit(X_train,y_train)
print(classifier.theta_)
print(classifier.sigma_)

In [ ]:
# Predicting the training set results
y_pred = classifier.predict(X_train)
cm = confusion_matrix(y_train, y_pred)
print(cm)
print(accuracy_score(y_train,y_pred))

In [ ]:
# Predicting the test set results
y_pred = classifier.predict(X_test,)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test,y_pred))

### Creating new input

In [ ]:
point = np.array([2,1,3,1],ndmin=2)

### Custom implementation

In [ ]:
# Computing sklearn model output to compare
classifier.predict_proba(point)

#### Building mean and std matrices
Notice here we are using list comprehension. Notice we work on the training set only. We build a mean and a std matrix where each row represent the values to define the likelihood pdf for each class.

In [ ]:
mean = np.array([X_train[y_train == i,:].mean(axis=0) for i in np.unique(y_train)])
mean

In [ ]:
std = np.array([X_train[y_train == i,:].std(axis=0) for i in np.unique(y_train)])
std

#### Building priors vector
We use frequency analysis to determine this. Notice we work on the training set only. We get one value per class.

In [ ]:
prior = [(y_train == i).sum()/len(y_train) for i in np.unique(y_train)]
prior

#### Computing the likelihood
We compute the likelihood of observing each feature value in the given input (this uses the proper pdf, that is the proper mean and std from the matrix). The result is a matrix the same size as the mean and std matrices.

In [ ]:
likelihood = norm.pdf(point,mean,std)
likelihood

#### Computing the log-likelihood instead
We compute the log likelihood. As you can see the values are not as small as the ones from the likelihood so we have less chance to underflow.

In [ ]:
loglikelihood = np.log(likelihood)
loglikelihood

#### Computing the posterior
Here we first use the straight foward method and along the way we can observe how we get really small values and of course a bigger chance to underflow.

In [ ]:
numerator = likelihood.prod(axis=1) * prior
numerator

In [ ]:
lognumerator = loglikelihood.sum(axis=1) + np.log(prior)
lognumerator

In [ ]:
posterior = numerator/numerator.sum()
posterior

In [ ]:
posterior.sum()

#### Computing the log-posterior
We compute the log posterior using two different ways to determine P(x). First we compute this in an 'unsafe' (less stable) way and then we use te logsumexp trick.

In [ ]:
numerator.sum()

In [ ]:
logposterior = lognumerator - np.log(numerator.sum()) # unsafe
logposterior

In [ ]:
posterior = np.exp(logposterior)
posterior

In [ ]:
posterior.sum()

In [ ]:
logposterior = lognumerator - logsumexp(lognumerator) # more stable
logposterior 

In [ ]:
posterior = np.exp(logposterior)
posterior

In [ ]:
posterior.sum()